In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [197]:
# Carga de tabla con explicaciones de cada columna

col_explicacion = pd.read_csv('descriptions_for_ column.csv', encoding='latin-1')
col_explicacion = col_explicacion.drop(columns=['Col_names_english', 
                                                'Descriptions_english'])
print(col_explicacion)

      Col_names_spanish                               Descriptions_spanish
0     Fecha de Registro                     Fecha de registro del divorcio
1                  Tipo  Necesario cuando alguna de las partes no acept...
2          Nacionalidad                        Nacionalidad del divorciado
3             F_Naci_Do                 Fecha de nacimiento del divorciado
4            LocNaci_Do             Localidad de nacimiento del divorciado
5           MpioNaci_Do             Municipio de nacimiento del divorciado
6           EFedNaci_Do    Entidad Federativa de nacimiento del divorciado
7           PaisNaci_Do                 PaÃ­s de nacimiento del divorciado
8                  Edad                                Edad del divorciado
9            MpioHab_Do   Municipio del domicilio que habita el divorciado
10           EFedHab_Do  Entidad federativa del domicilio que habita el...
11           PaisHab_Do    PaÃ­s del domicilio en que habita el divorciado
12            ElIngreso  

In [198]:
# Carga de DF + renombre de columnas + descarte de columnas 

df = pd.read_csv('divorces_2000-2015_original.csv', 
                 parse_dates=['Fecha de Registro', 'F_Naci_Do', 'F_Naci_Da', 'Fecha'],
                 date_format='%d-%m-%Y')

df = df.rename(columns={'Ocupaci√≥n_Do': 'Ocupacion_Do', 
                   'Ocupaci√≥n_Da': 'Ocupacion_Da',
                   'Matri_A√±os': 'Matri_Años',
                   'Edad': 'Edad_Do', 'Edad2': 'Edad_Da',
                   'Nacionalidad': 'Nacionalidad_Do', 
                   'Nacionalidad2': 'Nacionalidad_Da',
                   'Fecha': 'Fecha_matri', 'Fecha de Registro': 'Fecha_divor'})

df = df.drop(columns=['LocNaci_Do', 'MpioNaci_Do', 'EFedNaci_Do', 'MpioHab_Do',
                      'EFedHab_Do', 'LocHabi_Do', 'LocNaci_Da', 'MpioNaci_Da', 
                      'EFedNaci_Da', 'MpioHab_Da','EFedHab_Da', 'LocHabi_Da',
                      'Fecha de Registro 2', 'Localidad', 'Municipio'])

display(df.columns)

Index(['Fecha_divor', 'Tipo', 'Nacionalidad_Do', 'F_Naci_Do', 'PaisNaci_Do',
       'Edad_Do', 'PaisHab_Do', 'ElIngreso', 'Ocupacion_Do', 'Nacionalidad_Da',
       'F_Naci_Da', 'PaisNaci_Da', 'Edad_Da', 'PaisHab_Da', 'Ocupacion_Da',
       'EllaIngreso', 'Fecha_matri', 'Entidad Federativa', 'ElEscolaridad',
       'ElTrabajo', 'EllaEscolaridad', 'EllaTrabajo', 'Matri_Años',
       'Matri_Meses', 'Num_Hijos', 'Custodia'],
      dtype='object')

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Fecha_divor         4923 non-null   object 
 1   Tipo                4923 non-null   object 
 2   Nacionalidad_Do     4922 non-null   object 
 3   F_Naci_Do           4542 non-null   object 
 4   PaisNaci_Do         4796 non-null   object 
 5   Edad_Do             4816 non-null   float64
 6   PaisHab_Do          4599 non-null   object 
 7   ElIngreso           3504 non-null   float64
 8   Ocupacion_Do        4394 non-null   object 
 9   Nacionalidad_Da     4920 non-null   object 
 10  F_Naci_Da           4471 non-null   object 
 11  PaisNaci_Da         4784 non-null   object 
 12  Edad_Da             4772 non-null   float64
 13  PaisHab_Da          4618 non-null   object 
 14  Ocupacion_Da        4345 non-null   object 
 15  EllaIngreso         2804 non-null   float64
 16  Fecha_

In [200]:
df.describe()

,Edad_Do,ElIngreso,Edad_Da,EllaIngreso,Matri_Años,Matri_Meses,Num_Hijos
count,4816.000000,3.504000e+03,4772.000000,2804.000000,4688.000000,1555.000000,3011.000000
mean,39.443729,1.092011e+04,36.958089,7374.254094,11.723549,6.293891,1.815344
std,10.396888,7.056976e+04,9.934723,16337.048502,9.298888,3.865557,0.930711
min,19.000000,2.400000e+00,17.000000,3.500000,1.000000,0.000000,1.000000
25%,32.000000,3.000000e+03,29.000000,3000.000000,4.000000,4.000000,1.000000
50%,38.000000,5.600000e+03,35.000000,5000.000000,9.000000,6.000000,2.000000
75%,46.000000,1.000000e+04,43.000000,8000.000000,17.000000,9.000000,2.000000
max,91.000000,3.150242e+06,84.000000,708652.000000,61.000000,93.000000,10.000000


In [201]:
df['PaisNaci_Do'] = df['PaisNaci_Do'].fillna(df['PaisNaci_Do'].mode()[0])
df['PaisNaci_Da'] = df['PaisNaci_Da'].fillna(df['PaisNaci_Da'].mode()[0])

df['Nacionalidad_Do'] = df['Nacionalidad_Do'].fillna(df['Nacionalidad_Do'].mode()[0])
df['Nacionalidad_Da'] = df['Nacionalidad_Da'].fillna(df['Nacionalidad_Da'].mode()[0])

df['PaisHab_Do'] = df['PaisHab_Do'].fillna(df['PaisHab_Do'].mode()[0])
df['PaisHab_Da'] = df['PaisHab_Da'].fillna(df['PaisHab_Da'].mode()[0])

display(df['F_Naci_Do'])

0       18/12/75
1            NaN
2        22/2/55
3        20/1/64
4       30/10/75
          ...   
4918    29/10/77
4919    10/11/82
4920    28/10/54
4921      7/2/58
4922     22/6/60
Name: F_Naci_Do, Length: 4923, dtype: object

In [202]:
df.isnull().sum()

Fecha_divor              0
Tipo                     0
Nacionalidad_Do          0
F_Naci_Do              381
PaisNaci_Do              0
Edad_Do                107
PaisHab_Do               0
ElIngreso             1419
Ocupacion_Do           529
Nacionalidad_Da          0
F_Naci_Da              452
PaisNaci_Da              0
Edad_Da                151
PaisHab_Da               0
Ocupacion_Da           578
EllaIngreso           2119
Fecha_matri              0
Entidad Federativa       0
ElEscolaridad          304
ElTrabajo              356
EllaEscolaridad        380
EllaTrabajo            417
Matri_Años             235
Matri_Meses           3368
Num_Hijos             1912
Custodia              2851
dtype: int64